In [1]:
# 라이브러리 불러오기
import pandas as pd
# 경고창 무시 
import warnings
warnings.filterwarnings("ignore")

# 1. 상장폐지기업정리

In [2]:
# 2021년까지 코스피에 상장된 모든 기업 불러오기
상장폐지_df = pd.read_csv('./datasets/ts2000/코스피2021년까지상페기업.csv', encoding='cp949')
상장폐지_df

,회사명,거래소코드,회계년도,상장폐지일,소속코드,업종코드
0,(주)DB하이텍,990,2011/12,NaN,1,1
1,(주)DB하이텍,990,2012/12,NaN,1,1
2,(주)DB하이텍,990,2013/12,NaN,1,1
3,(주)DB하이텍,990,2014/12,NaN,1,1
4,(주)DB하이텍,990,2015/12,NaN,1,1
...,...,...,...,...,...,...
8058,흥아해운(주),3280,2017/12,NaN,1,1
8059,흥아해운(주),3280,2018/12,NaN,1,1
8060,흥아해운(주),3280,2019/12,NaN,1,1
8061,흥아해운(주),3280,2020/12,NaN,1,1


In [3]:
상장폐지_df[상장폐지_df['회계년도'] <= '2019/12']['회사명'].nunique()

793

In [4]:
# Dart에서 직접 확인한 데이터를 메모장에 저장하여 불러오는 코드
상장폐지사유_df = pd.read_csv('./상장폐지사유.txt', sep=',' , engine='python')
상장폐지사유_df.reset_index(inplace=True)
상장폐지사유_df.columns = ['회사명', '상장폐지사유']
상장폐지사유_df

,회사명,상장폐지사유
0,(주)다함이텍,자본잠식
1,(주)지아이바이오,감사의견거절
2,(주)금빛,해산사유발생
3,(주)나노트로닉스,기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당
4,(주)네오퍼플,감사의견거절
...,...,...
270,한화화인케미칼(주),피합병
271,현대정보기술(주),피합병
272,현대페인트(주),감사의견거절
273,현대하이스코(주),피합병


In [5]:
# "(주)"를 제거하는 함수 정의
def remove_joo(text):
    if '(주)' in text:
        return text.replace('(주)', '')
    else:
        return text

# '회사명' 컬럼에 있는 "(주)" 제거
상장폐지_df['회사명'] = 상장폐지_df['회사명'].apply(remove_joo)
상장폐지사유_df['회사명'] = 상장폐지사유_df['회사명'].apply(remove_joo)

In [6]:
# 코스닥 상장폐지된 기업 확인
상장폐지_df = 상장폐지_df[~상장폐지_df['상장폐지일'].isna()][['회사명','거래소코드','상장폐지일']].drop_duplicates()
상장폐지_df

,회사명,거래소코드,상장폐지일
313,넥솔론,110570,2017/04/17
407,다함이텍,9280,2013/04/16
453,대성합동지주,5620,2017/08/25
676,동성화학,5190,2021/04/16
708,동양건설산업,5900,2014/04/25
...,...,...,...
7587,한솔아트원제지,7190,2017/03/15
7676,한일건설,6440,2013/04/16
7770,한화화인케미칼,25850,2016/03/15
7961,현대페인트,11720,2016/11/21


In [7]:
# 상장폐지 데이터와 상장폐지사유 데이터를 회사명을 기준으로 merge
상장폐지사유merge_df = pd.merge(상장폐지_df, 상장폐지사유_df, how='left', on="회사명")
상장폐지사유merge_df

,회사명,거래소코드,상장폐지일,상장폐지사유
0,넥솔론,110570,2017/04/17,해산사유발생
1,다함이텍,9280,2013/04/16,자본잠식
2,대성합동지주,5620,2017/08/25,피합병
3,동성화학,5190,2021/04/16,피합병
4,동양건설산업,5900,2014/04/25,감사의견거절 및 자본전액잠식
...,...,...,...,...
64,한솔아트원제지,7190,2017/03/15,피합병
65,한일건설,6440,2013/04/16,자본전액잠식
66,한화화인케미칼,25850,2016/03/15,피합병
67,현대페인트,11720,2016/11/21,감사의견거절


In [8]:
# 공백제거
상장폐지사유merge_df['상장폐지사유'] = 상장폐지사유merge_df['상장폐지사유'].str.strip()

In [9]:
# 상장폐지사유 결측치 확인
상장폐지사유merge_df['상장폐지사유'].isna().sum()

0

In [10]:
# 상장폐지사유들 확인해보기
상장폐지사유merge_df['상장폐지사유'].value_counts()

피합병                               34
감사의견거절                            14
자진상장폐지                             4
자본잠식 50%이상 또는 매출액 50억원 미만 사실발생     4
최종부도발생                             2
횡령 배임                              2
자본전액잠식                             2
해산사유발생                             1
자본잠식                               1
감사의견거절 및 자본전액잠식                    1
액면가의 100분의 20미만인 상태로 61거래일이 경과     1
파산                                 1
상장폐지신청                             1
시가총액요건미달                           1
Name: 상장폐지사유, dtype: int64

In [11]:
for idx, row in 상장폐지사유merge_df.iterrows():
    if '감사' in row['상장폐지사유']:
        상장폐지사유merge_df.at[idx, '상장폐지사유'] = '감사의견거절'
for idx, row in 상장폐지사유merge_df.iterrows():
    if '자본' in row['상장폐지사유']:
        상장폐지사유merge_df.at[idx, '상장폐지사유'] = '자본잠식'
for idx, row in 상장폐지사유merge_df.iterrows():
    if '해산' in row['상장폐지사유']:
        상장폐지사유merge_df.at[idx, '상장폐지사유'] = '파산'
for idx, row in 상장폐지사유merge_df.iterrows():
    if '파산' in row['상장폐지사유']:
        상장폐지사유merge_df.at[idx, '상장폐지사유'] = '파산'
for idx, row in 상장폐지사유merge_df.iterrows():
    if '보고서' in row['상장폐지사유']:
        상장폐지사유merge_df.at[idx, '상장폐지사유'] = '보고서미제출'
for idx, row in 상장폐지사유merge_df.iterrows():
    if '피합' in row['상장폐지사유']:
        상장폐지사유merge_df.at[idx, '상장폐지사유'] = '피합병'
for idx, row in 상장폐지사유merge_df.iterrows():
    if '신청' in row['상장폐지사유']:
        상장폐지사유merge_df.at[idx, '상장폐지사유'] = '자진상장폐지'
for idx, row in 상장폐지사유merge_df.iterrows():
    if '자진' in row['상장폐지사유']:
        상장폐지사유merge_df.at[idx, '상장폐지사유'] = '자진상장폐지'

In [12]:
# 상장폐지 정의에 따른 1, 0 분류 코드
def change(x):
    if x == '피합병' or x == '자진상장폐지' or x == '코스피상장' or x == '코넥스':
        return 0
    else:
        return 1

상장폐지사유merge_df['상장폐지'] = 상장폐지사유merge_df['상장폐지사유'].apply(change)

In [13]:
# 상장폐지 기업 확인
상장폐지사유merge_df['상장폐지'].value_counts()

0    39
1    30
Name: 상장폐지, dtype: int64

### 2022년도까지 상장폐지 기업수 : 69개
### 부도정의로 분류한 결과 1: 30개 0: 39개

# 2. 연결데이터, 개별데이터 결합

In [14]:
# 성장성
성장성_연결 = pd.read_csv("./datasets/ts2000/kospi_성장성_연결.csv", encoding='cp949')
# 수익성
수익성_연결 = pd.read_csv("./datasets/ts2000/kospi_수익성_연결.csv", encoding='cp949')
# 안정성
안정성_연결 = pd.read_csv("./datasets/ts2000/kospi_안정성_연결.csv", encoding='cp949')
# 활동성
활동성_연결 = pd.read_csv("./datasets/ts2000/kospi_활동성_연결.csv", encoding='cp949')

In [15]:
# 컬럼명 변경
수익성_연결.rename(columns={'회사명' : '회사명1', '거래소코드' : '거래소코드1', '회계년도' : '회계년도1'}, inplace=True)
안정성_연결.rename(columns={'회사명' : '회사명2', '거래소코드' : '거래소코드2', '회계년도' : '회계년도2'}, inplace=True)
활동성_연결.rename(columns={'회사명' : '회사명3', '거래소코드' : '거래소코드3', '회계년도' : '회계년도3'}, inplace=True)

In [16]:
# 성장, 수익, 안정, 활동 결합
kospi_연결 = pd.concat([성장성_연결, 수익성_연결, 안정성_연결, 활동성_연결], axis=1)
kospi_연결.drop(['회사명1', '거래소코드1', '회계년도1','회사명2', '거래소코드2', '회계년도2','회사명3', '거래소코드3', '회계년도3'], axis=1, inplace=True)

In [17]:
# 컬럼명 정리 ( 특수문자제거 )
kospi_연결.columns = kospi_연결.columns.str.replace('연결','')
kospi_연결.columns = kospi_연결.columns.str.replace('원','')
kospi_연결.columns = kospi_연결.columns.str.replace('IFRS','')
kospi_연결.columns = kospi_연결.columns.str.replace('(','')
kospi_연결.columns = kospi_연결.columns.str.replace(')','')
kospi_연결.columns = kospi_연결.columns.str.replace(' ','')

In [18]:
# 연결재무제표 결측치 확인
kospi_연결.isna().sum()

회사명                  0
거래소코드                0
회계년도                 0
총자본증가율            1679
유형자산증가율           1679
투자부동산증가율          1679
비유동자산증가율          1679
유동자산증가율           1679
재고자산증가율           1679
자기자본증가율           1679
매출액증가율            1679
순이익증가율            1679
정상영업이익증가율         1679
매출액총이익률           1679
매출액순이익률           1679
총자본순이익률           1679
자기자본순이익률          1679
매출가대매출액비율         1679
영업비용대영업수익비율       1679
광고선전비대매출액비율       1679
기업순이익률            1679
R&D투자효율           1679
금융비용대총비용비율        1679
이자부담률             1679
1주당매출액            1679
1주당순이익            1679
1주당CASHFLOW       1679
1주당정상영업이익         1679
유동자산구성비율          1679
재고자산대유동자산비율       1679
유동자산대비유동자산비율      1679
당좌자산구성비율          1679
비유동자산구성비율         1679
자기자본구성비율          1679
타인자본구성비율          1679
비유동비율             1679
유동비율              1679
당좌비율              1679
현금비율              1678
매입채무대재고자산비율       1679
부채비율              1679
유동부채비율            1679
단기차입금대총차입금비율      1679
순운전자본비율    

## 연결 결측치 1679개

In [19]:
# 성장성
성장성_개별 = pd.read_csv("./datasets/ts2000/kospi_성장성_개별.csv", encoding='cp949')
# 수익성
수익성_개별 = pd.read_csv("./datasets/ts2000/kospi_수익성_개별.csv", encoding='cp949')
# 안정성
안정성_개별 = pd.read_csv("./datasets/ts2000/kospi_안정성_개별.csv", encoding='cp949')
# 활동성
활동성_개별 = pd.read_csv("./datasets/ts2000/kospi_활동성_개별.csv", encoding='cp949')

In [20]:
# 컬럼명 변경
수익성_개별.rename(columns={'회사명' : '회사명1', '거래소코드' : '거래소코드1', '회계년도' : '회계년도1'}, inplace=True)
안정성_개별.rename(columns={'회사명' : '회사명2', '거래소코드' : '거래소코드2', '회계년도' : '회계년도2'}, inplace=True)
활동성_개별.rename(columns={'회사명' : '회사명3', '거래소코드' : '거래소코드3', '회계년도' : '회계년도3'}, inplace=True)

In [21]:
# 성장, 수익, 안정, 활동 결합
kospi_개별 = pd.concat([성장성_개별, 수익성_개별, 안정성_개별, 활동성_개별], axis=1)
kospi_개별.drop(['회사명1', '거래소코드1', '회계년도1','회사명2', '거래소코드2', '회계년도2','회사명3', '거래소코드3', '회계년도3'], axis=1, inplace=True)

In [22]:
# 컬럼명 정리 ( 특수문자제거 )
kospi_개별.columns = kospi_개별.columns.str.replace('연결','')
kospi_개별.columns = kospi_개별.columns.str.replace('원','')
kospi_개별.columns = kospi_개별.columns.str.replace('IFRS','')
kospi_개별.columns = kospi_개별.columns.str.replace('(','')
kospi_개별.columns = kospi_개별.columns.str.replace(')','')
kospi_개별.columns = kospi_개별.columns.str.replace(' ','')

In [23]:
kospi_개별.isna().sum()

회사명                 0
거래소코드               0
회계년도                0
총자본증가율            239
유형자산증가율           239
투자부동산증가율          239
비유동자산증가율          239
유동자산증가율           239
재고자산증가율           239
자기자본증가율           239
매출액증가율            239
순이익증가율            239
정상영업이익증가율         239
매출액총이익률           239
매출액순이익률           239
총자본순이익률           239
자기자본순이익률          239
매출가대매출액비율         239
영업비용대영업수익비율       239
광고선전비대매출액비율       239
기업순이익률            239
R&D투자효율           239
금융비용대총비용비율        239
이자부담률             239
1주당매출액            239
1주당순이익            239
1주당CASHFLOW       239
1주당정상영업이익         239
유동자산구성비율          239
재고자산대유동자산비율       239
유동자산대비유동자산비율      239
당좌자산구성비율          239
비유동자산구성비율         239
자기자본구성비율          239
타인자본구성비율          239
비유동비율             239
유동비율              239
당좌비율              239
현금비율              238
매입채무대재고자산비율       239
부채비율              239
유동부채비율            239
단기차입금대총차입금비율      239
순운전자본비율           239
차입금비율             239
CASHFLOW대부

## 개별 결측치 239개

In [24]:
filled_kospi = kospi_연결.combine_first(kospi_개별)

In [25]:
filled_kospi

,회사명,거래소코드,회계년도,총자본증가율,유형자산증가율,투자부동산증가율,비유동자산증가율,유동자산증가율,재고자산증가율,자기자본증가율,...,총자본회전률,자기자본회전률,타인자본회전률,매입채무회전률,유동자산회전률,당좌자산회전률,재고자산회전률,매출채권회전률,유형자산회전율,순운전자본회전률
0,(주)DB하이텍,990,2011/12,-13.19,-7.89,0.00,-14.10,-10.08,-3.28,-22.60,...,0.43,1.56,0.59,20.51,1.87,2.54,7.04,6.12,0.70,0.72
1,(주)DB하이텍,990,2012/12,-3.44,-9.69,0.00,-7.69,10.52,2.79,-5.57,...,0.50,1.97,0.67,28.86,2.01,2.73,7.56,8.08,0.82,0.61
2,(주)DB하이텍,990,2013/12,-8.58,-7.20,0.00,-1.55,-27.88,-8.69,-29.61,...,0.45,1.99,0.57,20.81,1.86,2.60,6.51,7.74,0.75,0.63
3,(주)DB하이텍,990,2014/12,-6.95,-6.96,0.00,-11.75,11.04,-4.42,-41.05,...,0.56,3.49,0.66,17.22,2.41,3.45,8.02,8.06,0.92,0.86
4,(주)DB하이텍,990,2015/12,-0.02,-9.91,0.00,-12.26,36.42,20.64,110.48,...,0.68,3.56,0.84,16.56,2.28,3.08,8.73,7.67,1.19,1.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8794,흥아해운(주),3280,2018/12,-5.33,-12.07,70.56,-5.17,-6.33,-5.85,-37.42,...,0.91,7.93,1.03,11.87,6.73,8.14,38.87,19.15,1.43,1.75
8795,흥아해운(주),3280,2019/12,-46.38,-51.51,-97.68,-51.51,-13.42,-58.98,-57.73,...,0.17,1.96,0.18,2.41,1.01,1.16,7.70,4.78,0.28,0.28
8796,흥아해운(주),3280,2020/12,-33.46,-24.76,755.99,-32.47,-37.01,-2.10,-185.01,...,0.25,0.00,0.25,6.21,1.16,1.29,11.65,41.95,0.42,0.35
8797,흥아해운(주),3280,2021/12,-13.77,-20.91,10.39,-15.66,-6.48,-9.66,0.00,...,0.31,0.00,0.36,8.08,1.43,1.63,11.37,29.24,0.51,0.56


In [26]:
filled_kospi.isna().sum()

회사명                 0
거래소코드               0
회계년도                0
총자본증가율            214
유형자산증가율           214
투자부동산증가율          214
비유동자산증가율          214
유동자산증가율           214
재고자산증가율           214
자기자본증가율           214
매출액증가율            214
순이익증가율            214
정상영업이익증가율         214
매출액총이익률           214
매출액순이익률           214
총자본순이익률           214
자기자본순이익률          214
매출가대매출액비율         214
영업비용대영업수익비율       214
광고선전비대매출액비율       214
기업순이익률            214
R&D투자효율           214
금융비용대총비용비율        214
이자부담률             214
1주당매출액            214
1주당순이익            214
1주당CASHFLOW       214
1주당정상영업이익         214
유동자산구성비율          214
재고자산대유동자산비율       214
유동자산대비유동자산비율      214
당좌자산구성비율          214
비유동자산구성비율         214
자기자본구성비율          214
타인자본구성비율          214
비유동비율             214
유동비율              214
당좌비율              214
현금비율              213
매입채무대재고자산비율       214
부채비율              214
유동부채비율            214
단기차입금대총차입금비율      214
순운전자본비율           214
차입금비율             214
CASHFLOW대부

## 연결재무제표 결측치 개별재무제표 값으로 대체 후 남은 결측치 약 214개

-------------------------------------------------------

## 3. 생애주기, 업력 파생변수 결합

In [27]:
kospi_life = pd.read_csv('./datasets/ts2000/new_lifetime_kospi.csv', index_col=0)
kospi_life = kospi_life.iloc[:,3:]
kospi_concat = pd.concat([filled_kospi, kospi_life], axis=1)

In [28]:
kospi_history = pd.read_csv('./datasets/ts2000/kospi_설립연도상장연도.csv', index_col=0)
kospi_history= kospi_history.iloc[:,3:]
kospi_concat_2 = pd.concat([kospi_concat, kospi_history], axis=1)

In [29]:
상장기간 = pd.read_csv('./datasets/kospi_상장기간.csv',index_col=0)
kospi_concat_2['상장기간'] = 상장기간['상장기간']

## 4. 상장폐지기업 결합

In [30]:
상장폐지사유merge_df = 상장폐지사유merge_df[['거래소코드','상장폐지일', '상장폐지사유', '상장폐지']]
상장폐지사유merge_df['상장폐지년도'] = pd.to_datetime(상장폐지사유merge_df['상장폐지일'], format='%Y/%m/%d').dt.strftime('%Y')
상장폐지사유merge_df['상장폐지년도'] = 상장폐지사유merge_df['상장폐지년도'].astype(int)
상장폐지사유merge_df

,거래소코드,상장폐지일,상장폐지사유,상장폐지,상장폐지년도
0,110570,2017/04/17,파산,1,2017
1,9280,2013/04/16,자본잠식,1,2013
2,5620,2017/08/25,피합병,0,2017
3,5190,2021/04/16,피합병,0,2021
4,5900,2014/04/25,감사의견거절,1,2014
...,...,...,...,...,...
64,7190,2017/03/15,피합병,0,2017
65,6440,2013/04/16,자본잠식,1,2013
66,25850,2016/03/15,피합병,0,2016
67,11720,2016/11/21,감사의견거절,1,2016


In [31]:
상장폐지사유merge_df['상장폐지'].value_counts()

0    39
1    30
Name: 상장폐지, dtype: int64

In [32]:
# 코스닥 데이터와 상장폐지데이터 merge
kospi = pd.merge(kospi_concat_2, 상장폐지사유merge_df, how='left', on='거래소코드')

In [33]:
# 상장폐지 결측치 0으로 대체 (정상기업)
kospi['상장폐지'] = kospi['상장폐지'].fillna(0)
kospi['상장폐지'] = kospi['상장폐지'].astype(int)

In [34]:
# 라벨이 1인 데이터수 확인
kospi['상장폐지'].value_counts()

0    8675
1     124
Name: 상장폐지, dtype: int64

In [35]:
# 라벨이 1인 기업수 확인
len(kospi[kospi['상장폐지'] == 1]['회사명'].unique())

30

## 5. 외국기업(거래소코드 앞자리가 9인 기업), 금융 기업, 12월 결산일이 아닌 데이터, 거래소코드가 2개인 기업 모두 제거

5-1) 외국 기업 (거래소코드 앞자리가 9인 기업)

지우는 이유 : 재무기준이 다르기 때문

In [36]:
# 종목코드를 6자리로 통일. ex) 990 -> 000990
def padding(x):
    x = str(x)
    return x.rjust(6, "0")
# True, False 반전
def func_not(x):
    return not x

In [37]:
 # 거래소코드 padding
kospi["거래소코드"] = kospi["거래소코드"].map(padding)

In [38]:
# 외국계 기업 확인
kospi[kospi['거래소코드'].str[:1] == '9']['회사명'].unique()

array(['엘브이엠씨홀딩스(주)', '연합과기공고유한공사', '중국고섬공고유한공사', '중국원양자원유한공사',
       '평산차업집단유한공사'], dtype=object)

In [39]:
## 외국계 기업 제거
def func_isnt_fcompany(x):
    if x[:1] == '9':
        return False
    else:
        return True

print("제거할 외국계기업 데이터 수 : ", len(kospi[kospi["거래소코드"].map(func_isnt_fcompany).map(func_not)]))
kospi = kospi[kospi["거래소코드"].map(func_isnt_fcompany)]
kospi.shape # 외국계기업 데이터 25개 제거

제거할 외국계기업 데이터 수 :  25


(8774, 73)

5-2) 금융 기업 데이터 제거

지우는 이유 : 재무기준이 다르기 때문

In [40]:
# 금융 및 보험업 상장폐지 기업 확인
kospi[kospi['통계청 한국표준산업분류 10차(대분류)'].str.contains('금융 및 보험업')]['회사명'].unique()

array(['(주)경동인베스트', '(주)에이플러스에셋어드바이저', '(주)오리온홀딩스', '(주)카카오페이', '(주)효성',
       '미원홀딩스(주)', '에이치디씨(주)', '쿠쿠홀딩스(주)'], dtype=object)

In [41]:
# 금융 및 보험업 데이터 제거
print("제거할 금융 및 보험업 데이터 수 : ", len(kospi[kospi['통계청 한국표준산업분류 10차(대분류)'].str.contains('금융 및 보험업')]))
kospi = kospi[~kospi['통계청 한국표준산업분류 10차(대분류)'].str.contains('금융 및 보험업')]
kospi.shape # 금융 및 보험업 데이터 89개 제거

제거할 금융 및 보험업 데이터 수 :  89


(8685, 73)

5-3) 결산일이 12월이 아닌 데이터 제거

지우는 이유 : 비교 기준이 맞지 않기 때문

In [42]:
# 회계년도컬럼에서 월만 추출
kospi['회계년도월'] = pd.to_datetime(kospi['회계년도'], format='%Y/%m').dt.strftime('%m')
kospi['회계년도월'] = kospi['회계년도월'].astype(int)

In [43]:
# 회계년도가 12월이 아닌 데이터 제거
print("제거할 회계년도 12월이 아닌 데이터 수 : ", len(kospi[kospi['회계년도월'] != 12]))
kospi = kospi[kospi['회계년도월'] == 12]
kospi.drop(columns='회계년도월',axis=1, inplace=True)
kospi.shape # 회계년도가 12월이 아닌 데이터 47개 제거

제거할 회계년도 12월이 아닌 데이터 수 :  47


(8638, 73)

5-3) 거래소코드가 2개인 기업 제거

지우는 이유 : 데이터 통일

In [44]:
codecheck = kospi.groupby(['회사명'])['거래소코드'].nunique().reset_index()
codecheck.rename(columns = {'거래소코드':'거래소코드_개수'}, inplace=True)
codecheck = codecheck[codecheck['거래소코드_개수']>1]
codecheck

,회사명,거래소코드_개수
139,(주)신성이엔지,2
492,삼성물산(주),2
569,에스케이(주),2


In [45]:
## 회사수와 거래소코드수 차이가나는것 확인
print("회사수 : ", len(kospi["회사명"].unique()))
print("거래소코드수 : ", len(kospi["거래소코드"].unique()))

회사수 :  790
거래소코드수 :  793


In [46]:
## 거래소 코드 중복기업 전처리
list_overlap_code = list()
for i in list(kospi["회사명"].unique()):
    if len(kospi[kospi["회사명"]==i]["거래소코드"].unique()) > 1:
        list_overlap_code.append(i)
print("거래소코드가 2개인 기업 수:", len(list_overlap_code), "개")
# list_overlap_code

거래소코드가 2개인 기업 수: 3 개


In [47]:
# 중복 거래소코드 제거
def func_overlap_code(x):
    if x == (kospi[kospi["회사명"]==company_name]["거래소코드"].iloc[-1]) :
        return True
    else:
        return False

list_companyname = list(kospi["회사명"].unique())


dfs = list()


for i in list_companyname:
    company_name = i
    #각 기업별로 데이터프레임 생성
    df_concat = kospi[kospi["회사명"]==i]
    df_concat = df_concat[df_concat["거래소코드"].map(func_overlap_code)] # 거래소코드 바뀌기 전의 거래소코드의 데이터는 삭제
    dfs.append(df_concat)
kospi_1 = pd.concat(dfs)
kospi_1.reset_index(drop=True, inplace=True)

In [48]:
# 데이터 제거 전
print(kospi.shape)
# 데이터 제거 후 
print(kospi_1.shape) # 13개 데이터 제거

(8638, 73)
(8625, 73)


In [49]:
# 거래소코드가 2개인 기업 잘 제거됐는지 한번 더 확인
codecheck = kospi_1.groupby(['회사명'])['거래소코드'].nunique().reset_index()
codecheck.rename(columns = {'거래소코드':'거래소코드_개수'}, inplace=True)
codecheck = codecheck[codecheck['거래소코드_개수']>1]
codecheck

,회사명,거래소코드_개수


In [50]:
# 회사명이 2개인 거래소코드 존재 확인 
list_overlap_code = list()
for i in list(kospi_1["거래소코드"].unique()):
    if len(kospi_1[kospi_1["거래소코드"]==i]["회사명"].unique()) > 1:
        list_overlap_code.append(i)
list_overlap_code

[]

# 6. 성장성 데이터 계산한 값으로 대체

In [51]:
kospi_성장성 = pd.read_csv('./datasets/ts2000/kospi_성장율_계산_edit.csv', index_col=0)

In [52]:
kospi_성장성.reset_index(drop=True, inplace=True)

In [53]:
kospi_1 = kospi_1.iloc[:,13:]
kospi_1

,매출액총이익률,매출액순이익률,총자본순이익률,자기자본순이익률,매출가대매출액비율,영업비용대영업수익비율,광고선전비대매출액비율,기업순이익률,R&D투자효율,금융비용대총비용비율,...,기업생애주기,통계청 한국표준산업분류 10차(대분류),설립연도,상장연도,경영기간,상장기간,상장폐지일,상장폐지사유,상장폐지,상장폐지년도
0,16.62,-17.74,-7.59,-27.74,83.38,110.96,0.00,-3.57,0.0,7.77,...,성숙기,제조업,1953,1975,58,37,NaN,NaN,0,NaN
1,16.38,-5.31,-2.66,-10.47,83.62,102.64,0.00,1.25,0.0,6.90,...,성숙기,제조업,1953,1975,59,38,NaN,NaN,0,NaN
2,18.08,-16.82,-7.49,-33.51,81.92,101.95,0.00,-2.38,0.0,9.98,...,성숙기,제조업,1953,1975,60,39,NaN,NaN,0,NaN
3,24.11,-13.56,-7.54,-47.32,75.89,91.97,0.00,-1.10,0.0,11.38,...,성장기,제조업,1953,1975,61,40,NaN,NaN,0,NaN
4,34.46,19.01,12.87,67.63,65.54,81.25,0.00,17.82,0.0,7.85,...,성숙기,제조업,1953,1975,62,41,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8620,-0.93,-11.48,-10.47,-91.08,100.93,104.99,0.03,-6.58,0.0,3.63,...,쇠퇴기,운수 및 창고업,1961,1976,57,43,NaN,NaN,0,NaN
8621,5.54,-50.27,-8.32,-98.70,94.46,112.10,0.14,-4.70,0.0,10.55,...,쇠퇴기,운수 및 창고업,1961,1976,58,44,NaN,NaN,0,NaN
8622,8.26,-57.20,-14.18,-2195.07,91.74,106.52,0.02,-9.91,0.0,8.54,...,쇠퇴기,운수 및 창고업,1961,1976,59,45,NaN,NaN,0,NaN
8623,6.94,20.11,6.16,42.88,93.06,102.29,0.01,6.74,0.0,1.24,...,쇠퇴기,운수 및 창고업,1961,1976,60,46,NaN,NaN,0,NaN


In [54]:
# 계산된 성장성 데이터와 결합
df = pd.concat([kospi_성장성, kospi_1],axis=1)
df

,회사명,거래소코드,회계년도,총자본증가율,유형자산증가율,유동자산증가율,비유동자산증가율,재고자산증가율,자기자본증가율,매출액증가율,...,기업생애주기,통계청 한국표준산업분류 10차(대분류),설립연도,상장연도,경영기간,상장기간,상장폐지일,상장폐지사유,상장폐지,상장폐지년도
0,(주)DB하이텍,990,2011/12,-13.191447,-7.891077,-10.081970,-14.095356,-3.275049,-22.600599,-10.939080,...,성숙기,제조업,1953,1975,58,37,NaN,NaN,0,NaN
1,(주)DB하이텍,990,2012/12,-3.443493,-9.686758,10.516720,-7.691245,2.786819,-5.573229,6.984586,...,성숙기,제조업,1953,1975,59,38,NaN,NaN,0,NaN
2,(주)DB하이텍,990,2013/12,-8.582386,-7.200145,-27.877947,-1.553126,-8.685120,-29.610888,-16.424560,...,성숙기,제조업,1953,1975,60,39,NaN,NaN,0,NaN
3,(주)DB하이텍,990,2014/12,-6.945914,-6.957581,11.042738,-11.746871,-4.418470,-41.050611,14.980395,...,성장기,제조업,1953,1975,61,40,NaN,NaN,0,NaN
4,(주)DB하이텍,990,2015/12,-0.023658,-9.910153,36.422276,-12.262137,20.641804,110.481098,17.416569,...,성숙기,제조업,1953,1975,62,41,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8620,흥아해운(주),3280,2018/12,-5.331353,-12.065399,-6.328074,-5.174121,-5.851729,-37.418295,-9.870676,...,쇠퇴기,운수 및 창고업,1961,1976,57,43,NaN,NaN,0,NaN
8621,흥아해운(주),3280,2019/12,-46.377495,-51.510121,-13.423469,-51.512707,-58.974904,-57.730902,-86.447591,...,쇠퇴기,운수 및 창고업,1961,1976,58,44,NaN,NaN,0,NaN
8622,흥아해운(주),3280,2020/12,-33.456932,-24.762048,-37.005650,-32.469532,-2.112494,-185.009218,-12.947429,...,쇠퇴기,운수 및 창고업,1961,1976,59,45,NaN,NaN,0,NaN
8623,흥아해운(주),3280,2021/12,-13.765796,-20.906906,-6.482782,-15.656115,-9.651794,491.645107,-8.112302,...,쇠퇴기,운수 및 창고업,1961,1976,60,46,NaN,NaN,0,NaN


## 7. Labeling

In [55]:
# 인덱스 초기화
df.reset_index(inplace=True)

In [56]:
# 컬럼추출
df_label = df[['index','회사명', '거래소코드', '회계년도', '상장폐지일', '상장폐지']]
df_label

,index,회사명,거래소코드,회계년도,상장폐지일,상장폐지
0,0,(주)DB하이텍,990,2011/12,NaN,0
1,1,(주)DB하이텍,990,2012/12,NaN,0
2,2,(주)DB하이텍,990,2013/12,NaN,0
3,3,(주)DB하이텍,990,2014/12,NaN,0
4,4,(주)DB하이텍,990,2015/12,NaN,0
...,...,...,...,...,...,...
8620,8620,흥아해운(주),3280,2018/12,NaN,0
8621,8621,흥아해운(주),3280,2019/12,NaN,0
8622,8622,흥아해운(주),3280,2020/12,NaN,0
8623,8623,흥아해운(주),3280,2021/12,NaN,0


In [57]:
# 상장폐지된 기업 데이터만 추출
df_label= df_label[df_label['상장폐지'] == 1]
df_label= df_label.groupby('회사명')['index'].apply(list)

In [58]:
# 상장폐지된 기업의 인덱스만 추출
df_label = pd.DataFrame(df_label)
df_label

,index
회사명,
(주)넥솔론,"[330, 331, 332, 333, 334, 335]"
(주)다함이텍,"[432, 433]"
(주)동양건설산업,"[752, 753, 754]"
(주)로케트전기,"[887, 888, 889]"
(주)배명금속,[984]
(주)보루네오가구,"[1033, 1034, 1035, 1036, 1037, 1038]"
(주)신일건업,"[1594, 1595, 1596, 1597]"
(주)신한,"[1598, 1599, 1600, 1601, 1602, 1603, 1604, 160..."
(주)알앤엘바이오,"[1749, 1750]"


In [59]:
# 상장폐지가 됐던 년도(t)에 t-1년 데이터에 1을 주는 코드
for j in df_label.index:
    for i in df.index:
        if df_label["index"][j][-1] == i:
            df.loc[i,'label'] = 1
df['label'] = df["label"].fillna(0)

In [60]:
# target 인트형 변환
df['label'] = df['label'].astype(int)

In [61]:
# 최종 데이터 수 
df['label'].value_counts()

0    8599
1      26
Name: label, dtype: int64

In [62]:
# 불필요 컬럼 삭제 ( 파생변수, target만들기 위해 사용했던 변수 삭제 )
df.drop(columns=['상장폐지', 'index', '상장폐지일','상장폐지사유', '상장폐지년도'],axis=1,inplace=True)

In [63]:
# 코스닥 정상기업
print("정상기업수:",df[df['label'] == 0]['회사명'].nunique())
# 코스닥 정상기업 데이터
print("정상기업 데이터수:", len(df[df['label'] == 0]))
# 코스닥 상장폐지기업
print("상장폐지기업수:",df[df['label'] == 1]['회사명'].nunique())
# 코스닥 상장폐지기업 데이터
print("상장폐지기업 데이터수:", len(df[df['label'] == 1]))

정상기업수: 786
정상기업 데이터수: 8599
상장폐지기업수: 26
상장폐지기업 데이터수: 26


In [64]:
list = df[df['label']==1]['회사명'].tolist()

In [65]:
# 추출한 데이터를 저장할 빈 데이터프레임을 만듭니다.
result_df = pd.DataFrame()

# 리스트의 각 회사명과 데이터프레임의 회사명을 비교합니다.
for company in list:
    selected_data = df[df['회사명'] == company]
    result_df = pd.concat([result_df, selected_data])

# 결과 데이터프레임을 출력합니다.
index_list = result_df[result_df['label']== 0].index

In [66]:
# 추출한 데이터를 저장할 빈 데이터프레임을 만듭니다.
result_df = pd.DataFrame()

# 인덱스가 리스트에 포함되지 않은 데이터만 추출합니다.
for i in df.index:
    if i not in index_list:
        result_df = result_df.append(df.loc[i])

# 결과 데이터프레임을 출력합니다.
result_df

,회사명,거래소코드,회계년도,총자본증가율,유형자산증가율,유동자산증가율,비유동자산증가율,재고자산증가율,자기자본증가율,매출액증가율,...,투자활동으로인한현금흐름,재무활동으로인한현금흐름,현금및현금성자산의증가,기업생애주기,통계청 한국표준산업분류 10차(대분류),설립연도,상장연도,경영기간,상장기간,label
0,(주)DB하이텍,990,2011/12,-13.191447,-7.891077,-10.081970,-14.095356,-3.275049,-22.600599,-10.939080,...,-23348.0,-88776.0,-52752.0,성숙기,제조업,1953,1975,58,37,0
1,(주)DB하이텍,990,2012/12,-3.443493,-9.686758,10.516720,-7.691245,2.786819,-5.573229,6.984586,...,-49272.0,-22718.0,-107.0,성숙기,제조업,1953,1975,59,38,0
2,(주)DB하이텍,990,2013/12,-8.582386,-7.200145,-27.877947,-1.553126,-8.685120,-29.610888,-16.424560,...,-14426.0,-18090.0,-4695.0,성숙기,제조업,1953,1975,60,39,0
3,(주)DB하이텍,990,2014/12,-6.945914,-6.957581,11.042738,-11.746871,-4.418470,-41.050611,14.980395,...,-43635.0,1206.0,5147.0,성장기,제조업,1953,1975,61,40,0
4,(주)DB하이텍,990,2015/12,-0.023658,-9.910153,36.422276,-12.262137,20.641804,110.481098,17.416569,...,-12209.0,-71767.0,67764.0,성숙기,제조업,1953,1975,62,41,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8620,흥아해운(주),3280,2018/12,-5.331353,-12.065399,-6.328074,-5.174121,-5.851729,-37.418295,-9.870676,...,84962.0,-55273.0,-4697.0,쇠퇴기,운수 및 창고업,1961,1976,57,43,0
8621,흥아해운(주),3280,2019/12,-46.377495,-51.510121,-13.423469,-51.512707,-58.974904,-57.730902,-86.447591,...,117484.0,-107050.0,-1615.0,쇠퇴기,운수 및 창고업,1961,1976,58,44,0
8622,흥아해운(주),3280,2020/12,-33.456932,-24.762048,-37.005650,-32.469532,-2.112494,-185.009218,-12.947429,...,46646.0,-51848.0,-8745.0,쇠퇴기,운수 및 창고업,1961,1976,59,45,0
8623,흥아해운(주),3280,2021/12,-13.765796,-20.906906,-6.482782,-15.656115,-9.651794,491.645107,-8.112302,...,13201.0,23793.0,33276.0,쇠퇴기,운수 및 창고업,1961,1976,60,46,0


In [67]:
df = result_df.reset_index(drop=True)

In [68]:
# 코스닥 정상기업
print("정상기업수:",df[df['label'] == 0]['회사명'].nunique())
# 코스닥 정상기업 데이터
print("정상기업 데이터수:", len(df[df['label'] == 0]))
# 코스닥 상장폐지기업
print("상장폐지기업수:",df[df['label'] == 1]['회사명'].nunique())
# 코스닥 상장폐지기업 데이터
print("상장폐지기업 데이터수:", len(df[df['label'] == 1]))

정상기업수: 764
정상기업 데이터수: 8515
상장폐지기업수: 26
상장폐지기업 데이터수: 26


# 8. 상장일 이전 데이터 제거

In [69]:
# 상장일 데이터 로드 후 회사명기준으로 중복 데이터 삭제
df1 = pd.read_csv('./datasets/상장일.csv', encoding='cp949')
df1 = df1[['거래소코드', '상장일']]
df1.drop_duplicates('거래소코드', inplace=True)

In [70]:
df2 = pd.merge(df, df1, how='left', on='거래소코드')

In [71]:
df2['회계년도'] = df2['회계년도'].astype('object') + '/31'
df3 = df2[~(df2['상장일'] >= df2['회계년도'])]
df3['label'].value_counts()

0    8091
1      26
Name: label, dtype: int64

In [72]:
# 회계년도 다시 원래대로 초기화
df3['회계년도'] = pd.to_datetime(df3['회계년도'], format=('%Y/%m/%d')).dt.strftime('%Y/%m')

In [73]:
# 컬럼명 변경 ( 매출가대매출액비율 -> 매출원가대매출액비율 )
df3.rename(columns={'매출가대매출액비율' : '매출원가대매출액비율', '영업활동으로인한현금흐름' : '영업활동현금흐름',
                        '투자활동으로인한현금흐름' : '투자활동현금흐름', '재무활동으로인한현금흐름' : '재무활동현금흐름'}, inplace=True)

In [74]:
# 불필요 컬럼 삭제
df3.drop(columns=['광고선전비대매출액비율', 'R&D투자효율', '설립연도', '상장연도', '상장일'], axis=1, inplace=True)

In [75]:
df3['경영기간'] = df3['경영기간'].astype(float)
df3['상장기간'] = df3['상장기간'].astype(float)

# 9. 회사명이 2개이고 거래소코드가 1개인 기업 제거

In [76]:
# 제이콘텐트리 회사 제거
df4 = df3[df3['회사명'] != '(주)제이콘텐트리']

In [77]:
df4.reset_index(drop=True, inplace=True)

# 10. 22년도 데이터 제거

In [78]:
df5 = df4[df4['회계년도'] != '2022/12']
df5.reset_index(drop=True, inplace=True)

In [79]:
df5

,회사명,거래소코드,회계년도,총자본증가율,유형자산증가율,유동자산증가율,비유동자산증가율,재고자산증가율,자기자본증가율,매출액증가율,...,순운전자본회전률,영업활동현금흐름,투자활동현금흐름,재무활동현금흐름,현금및현금성자산의증가,기업생애주기,통계청 한국표준산업분류 10차(대분류),경영기간,상장기간,label
0,(주)DB하이텍,990,2011/12,-13.191447,-7.891077,-10.081970,-14.095356,-3.275049,-22.600599,-10.939080,...,0.72,60749.0,-23348.0,-88776.0,-52752.0,성숙기,제조업,58.0,37.0,0
1,(주)DB하이텍,990,2012/12,-3.443493,-9.686758,10.516720,-7.691245,2.786819,-5.573229,6.984586,...,0.61,73877.0,-49272.0,-22718.0,-107.0,성숙기,제조업,59.0,38.0,0
2,(주)DB하이텍,990,2013/12,-8.582386,-7.200145,-27.877947,-1.553126,-8.685120,-29.610888,-16.424560,...,0.63,28381.0,-14426.0,-18090.0,-4695.0,성숙기,제조업,60.0,39.0,0
3,(주)DB하이텍,990,2014/12,-6.945914,-6.957581,11.042738,-11.746871,-4.418470,-41.050611,14.980395,...,0.86,48078.0,-43635.0,1206.0,5147.0,성장기,제조업,61.0,40.0,0
4,(주)DB하이텍,990,2015/12,-0.023658,-9.910153,36.422276,-12.262137,20.641804,110.481098,17.416569,...,1.11,152722.0,-12209.0,-71767.0,67764.0,성숙기,제조업,62.0,41.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7384,흥아해운(주),3280,2017/12,-11.431335,-16.501054,13.291350,-14.378792,11.180988,-39.276938,0.562792,...,2.09,-6892.0,-2001.0,14484.0,5854.0,도입기,운수 및 창고업,56.0,42.0,0
7385,흥아해운(주),3280,2018/12,-5.331353,-12.065399,-6.328074,-5.174121,-5.851729,-37.418295,-9.870676,...,1.75,-33795.0,84962.0,-55273.0,-4697.0,쇠퇴기,운수 및 창고업,57.0,43.0,0
7386,흥아해운(주),3280,2019/12,-46.377495,-51.510121,-13.423469,-51.512707,-58.974904,-57.730902,-86.447591,...,0.28,-12048.0,117484.0,-107050.0,-1615.0,쇠퇴기,운수 및 창고업,58.0,44.0,0
7387,흥아해운(주),3280,2020/12,-33.456932,-24.762048,-37.005650,-32.469532,-2.112494,-185.009218,-12.947429,...,0.35,-3958.0,46646.0,-51848.0,-8745.0,쇠퇴기,운수 및 창고업,59.0,45.0,0


In [80]:
df5.to_csv('./datasets/1.kospi_preprocessing완료.csv')